In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import tkinter as tk
from tkinter import filedialog
import os
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

In [4]:
# --- 1. モデル関数とUIの定義 ---

# ご指定の数式に基づく、GTP増加モデル
def gtp_rise_model(t_sec, k_ex, initial_gtp_fraction, initial_gdp_fraction):
    """
    GTPの増加をフィッティングするモデル。
    initial_gdp_fraction はデータから取得した固定値。
    """
    return initial_gtp_fraction + initial_gdp_fraction * (1 - np.exp(-k_ex * t_sec))

# UI要素の作成
run_button = widgets.Button(description="ファイルを選択して解析実行", button_style='success', layout=widgets.Layout(width='300px'))
output_area = widgets.Output()
xmax_widget = widgets.IntText(description='グラフ横軸の最大値 (分):', value=400, style=style)

# --- 2. メインの処理関数 ---

def on_run_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        
        # --- ファイル選択 ---
        root = tk.Tk()
        root.withdraw()
        root.attributes('-topmost', True)
        file_path = filedialog.askopenfilename(
            title="kex解析用のCSVファイルを選択",
            filetypes=[("Excel and CSV files", "*.xlsx *.csv"), ("All files", "*.*")]
        )
        if not file_path:
            print("ファイルが選択されませんでした。")
            return
            
        xmax_minutes = xmax_widget.value

        # --- グラフの準備 ---
        plt.figure(figsize=(10, 6))
        
        try:
            if file_path.endswith('.csv'): df = pd.read_csv(file_path)
            else: df = pd.read_excel(file_path)
            
            filename = os.path.basename(file_path)
            print(f"Processing: {filename}\n")

            # --- データの前処理 ---
            required_columns = ['time_min', 'gdp_intensity', 'gtp_intensity']
            if not all(col in df.columns for col in required_columns):
                print(f"エラー: '{filename}' に必要な列 ('time_min', 'gdp_intensity', 'gtp_intensity') がありません。")
                return
            
            df.dropna(subset=required_columns, inplace=True)
            if len(df) < 3:
                print(f"エラー: '{filename}' にフィッティング可能なデータが不足しています。")
                return
            
            df['time_sec'] = df['time_min'] * 60
            
            # 規格化のための基準値 (t=0の合計強度)
            total_initial_intensity = df['gtp_intensity'].iloc[0] + df['gdp_intensity'].iloc[0]
            df['GTP_fraction'] = df['gtp_intensity'] / total_initial_intensity
            df['GDP_fraction'] = df['gdp_intensity'] / total_initial_intensity
            
            # --- フィッティング ---
            # モデルに必要な固定値を取得 (t=0でのGDPの割合)
            initial_gdp_frac = df['GDP_fraction'].iloc[0]
            
            # フィットさせる変数は k_ex と initial_gtp_fraction の2つ
            popt, pcov = curve_fit(
                lambda t, k_ex, initial_gtp_fraction: gtp_rise_model(t, k_ex, initial_gtp_fraction, initial_gdp_frac),
                df['time_sec'],
                df['GTP_fraction'],
                p0=[0.0001, df['GTP_fraction'].iloc[0]] # 初期値の推定
            )
            
            # --- 結果の表示 ---
            k_ex_fit, i_gtp0_fit = popt
            k_ex_err = np.sqrt(np.diag(pcov))[0]
            
            print("--- kex 解析結果 ---")
            print(f"  k_ex = {k_ex_fit:.2E} ± {k_ex_err:.1E} (s⁻¹)")
            print("--------------------")

            # --- グラフへの描画 ---
            plt.scatter(df['time_min'], df['GTP_fraction'], label=f'{filename} ')
            
            time_fit_min = np.linspace(0, xmax_minutes, 400)
            time_fit_sec = time_fit_min * 60
            
            # フィット曲線を描画
            fit_curve = gtp_rise_model(time_fit_sec, k_ex_fit, i_gtp0_fit, initial_gdp_frac)
            
            fit_label = (
                r'$[Ras_{GTP}](t) = [Ras_{GTP}]_0 + [Ras_{GDP}]_0(1 - e^{-k_{ex}t})$' + '\n'
                f'$k_{{ex}} = ${k_ex_fit:.2E} s⁻¹'
            )
            plt.plot(time_fit_min, fit_curve, linestyle='--', color='red', label=fit_label)

        except Exception as e:
            print(f"処理中にエラーが発生しました: {e}")
        
        # --- グラフの最終仕上げ ---
        plt.title('GDP-GTP Exchange Kinetics')
        plt.xlabel('Time (min)')
        plt.ylabel('GTP bound ratio')
        plt.legend(fontsize='small')
        plt.grid(True)
        plt.xlim(-5, xmax_minutes)
        plt.ylim(bottom=0) # y軸の最小値を0に設定
        plt.show()

# --- 3. UIの表示とボタンへの機能の紐付け ---
form = widgets.VBox([
    widgets.HTML(value="<h3>kex 解析 (GTP増加モデル)</h3>"),
    xmax_widget,
    run_button,
    output_area
])

run_button.on_click(on_run_button_clicked)
display(form)


NameError: name 'style' is not defined